In [1]:
import requests
import json
import time
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np

load_dotenv()

MLFLOW_ABS_DIR = os.getenv("MLFLOW_ABS_DIR")

In [2]:
os.getcwd()

'e:\\projects\\datasys\\ml_histdata\\experiments'

### 5min synthetic data

In [3]:
import pandas as pd
import numpy as np

# Sample DataFrame with initial dates
data = {
    "date": pd.date_range(start="2022-05-28", periods=5, freq="D", tz="UTC"),
    "pm1": [0.2, 0.5, 1.5, 3.0, 10.1],
    "pm10": [5.8, 8.0, 17.8, 28.2, 15.7],
    "pm25": [4.4, 11.5, 15.5, 21.8, 31.3],
    "rh": [51.0, 55.8, 60.7, 65.7, 70.8],
    "temp": [27.8, 28.1, 30.5, 33.0, 35.5],
}
df = pd.DataFrame(data)

# Get today's date
today = pd.Timestamp.utcnow().normalize()  # Get today at midnight UTC

# Get the last date in the dataset
last_date = df["date"].max().normalize()

# Create a full date range from the last date to today
full_date_range = pd.date_range(start=last_date, end=today, freq="D", tz="UTC")

# Use the last known values to generate data for missing days
new_rows = []
for date in full_date_range:
    # Pick the most recent available row (if we are extending beyond existing dates)
    row = df.iloc[-1]  # Use last row as reference
    
    # Generate 5-minute intervals for each day
    time_intervals = pd.date_range(start=date, periods=288, freq="5min", tz="UTC")
    
    for t in time_intervals:
        new_rows.append({
            "date": t,
            "pm1": row["pm1"] + np.random.uniform(0, 0.5),
            "pm10": row["pm10"] + np.random.uniform(0, 5),
            "pm25": row["pm25"] + np.random.uniform(0, 5),
            "rh": row["rh"] + np.random.uniform(0, 5),
            "temp": row["temp"] + np.random.uniform(0, 0.5),
        })

# Convert to DataFrame
synthetic_df = pd.DataFrame(new_rows)


In [4]:
synthetic_df.tail()

,date,pm1,pm10,pm25,rh,temp
298939,2025-04-03 23:35:00+00:00,10.497496,18.031095,31.723675,71.056931,35.945196
298940,2025-04-03 23:40:00+00:00,10.511469,19.537682,36.057689,75.321313,35.564277
298941,2025-04-03 23:45:00+00:00,10.147890,20.269658,33.674600,72.910324,35.852872
298942,2025-04-03 23:50:00+00:00,10.226699,15.845363,31.780512,75.573756,35.941513
298943,2025-04-03 23:55:00+00:00,10.290676,18.580922,36.008220,75.129386,35.778787


In [86]:
synthetic_df.to_csv('../historical_data/synthetic_hist_data.csv', index=False)

### Feature engineering

In [3]:
df = pd.read_csv('../historical_data/synthetic_hist_data.csv')
df.isna().sum().sum()

np.int64(0)

In [4]:
df.head()

,date,pm1,pm10,pm25,rh,temp
0,2022-06-01 00:00:00+00:00,10.155785,18.754484,31.445490,71.800804,35.776383
1,2022-06-01 00:05:00+00:00,10.274655,19.053324,33.004086,75.182165,35.923086
2,2022-06-01 00:10:00+00:00,10.329388,17.053652,31.626523,72.317361,35.654168
3,2022-06-01 00:15:00+00:00,10.167943,18.328607,33.213043,73.215120,35.576479
4,2022-06-01 00:20:00+00:00,10.121747,17.553675,34.435514,71.837578,35.993904


In [5]:

df['date'] = pd.to_datetime(df['date'])

In [6]:
df.head()

,date,pm1,pm10,pm25,rh,temp
0,2022-06-01 00:00:00+00:00,10.155785,18.754484,31.445490,71.800804,35.776383
1,2022-06-01 00:05:00+00:00,10.274655,19.053324,33.004086,75.182165,35.923086
2,2022-06-01 00:10:00+00:00,10.329388,17.053652,31.626523,72.317361,35.654168
3,2022-06-01 00:15:00+00:00,10.167943,18.328607,33.213043,73.215120,35.576479
4,2022-06-01 00:20:00+00:00,10.121747,17.553675,34.435514,71.837578,35.993904


In [7]:
# Extract time features
df["hour"] = df["date"].dt.hour  
df["day_of_week"] = df["date"].dt.dayofweek  
df["month"] = df["date"].dt.month  
df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int) 

# Interactions
df["temp_x_rh"] = df["temp"] * df["rh"]
df["pm1_x_pm10"] = df["pm1"] * df["pm10"]
df["pm1_x_pm25"] = df["pm1"] * df["pm25"]
df["pm10_x_pm25"] = df["pm10"] * df["pm25"]

In [8]:
df.head()

,date,pm1,pm10,pm25,rh,temp,hour,day_of_week,month,is_weekend,temp_x_rh,pm1_x_pm10,pm1_x_pm25,pm10_x_pm25
0,2022-06-01 00:00:00+00:00,10.155785,18.754484,31.445490,71.800804,35.776383,0,2,6,0,2568.773091,190.466500,319.353623,589.743946
1,2022-06-01 00:05:00+00:00,10.274655,19.053324,33.004086,75.182165,35.923086,0,2,6,0,2700.775347,195.766325,339.105595,628.837531
2,2022-06-01 00:10:00+00:00,10.329388,17.053652,31.626523,72.317361,35.654168,0,2,6,0,2578.415324,176.153783,326.682614,539.347728
3,2022-06-01 00:15:00+00:00,10.167943,18.328607,33.213043,73.215120,35.576479,0,2,6,0,2604.736209,186.364223,337.708314,608.748804
4,2022-06-01 00:20:00+00:00,10.121747,17.553675,34.435514,71.837578,35.993904,0,2,6,0,2585.714916,177.673859,348.547559,604.469827


In [9]:
df.columns

Index(['date', 'pm1', 'pm10', 'pm25', 'rh', 'temp', 'hour', 'day_of_week',
       'month', 'is_weekend', 'temp_x_rh', 'pm1_x_pm10', 'pm1_x_pm25',
       'pm10_x_pm25'],
      dtype='object')

In [10]:
import mlflow
import mlflow.pyfunc
import pandas as pd
from sklearn.ensemble import RandomForestRegressor  # Example model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

mlflow.set_tracking_uri(f"sqlite:///{MLFLOW_ABS_DIR}/mlflow.db")
mlflow.set_experiment("AQI_Forecasting")

2025/04/03 20:03:01 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/04/03 20:03:01 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='file:///e:/projects/datasys/ml_histdata/experiments/mlruns/1', creation_time=1743690786418, experiment_id='1', last_update_time=1743690786418, lifecycle_stage='active', name='AQI_Forecasting', tags={}>

In [11]:
df.columns

Index(['date', 'pm1', 'pm10', 'pm25', 'rh', 'temp', 'hour', 'day_of_week',
       'month', 'is_weekend', 'temp_x_rh', 'pm1_x_pm10', 'pm1_x_pm25',
       'pm10_x_pm25'],
      dtype='object')

In [12]:
# Define feature columns (excluding 'date' and 'pm25')
feature_cols = [
    'pm1', 'pm10', 'pm25', 'rh', 'temp', 'hour', 'day_of_week',
       'month', 'is_weekend', 'temp_x_rh', 'pm1_x_pm10', 'pm1_x_pm25',
       'pm10_x_pm25']

# Forecasting horizon (next 30 mins)
forecast_horizon = 6

# Create future target columns
for i in range(1, forecast_horizon + 1):
    df[f"pm25_t+{i}"] = df["pm25"].shift(-i)

# Drop rows with NaN (caused by shifting forward)
df.dropna(inplace=True)

# Define target columns (future values of pm25)
target_cols = [f"pm25_t+{i}" for i in range(1, forecast_horizon + 1)]

# Split data (80% train, 20% test)
train_size = int(len(df) * 0.8)
train, test = df.iloc[:train_size], df.iloc[train_size:]

X_train, y_train = train[feature_cols], train[target_cols]
X_test, y_test = test[feature_cols], test[target_cols]


In [13]:
X_train[:10]

,pm1,pm10,pm25,rh,temp,hour,day_of_week,month,is_weekend,temp_x_rh,pm1_x_pm10,pm1_x_pm25,pm10_x_pm25
0,10.155785,18.754484,31.445490,71.800804,35.776383,0,2,6,0,2568.773091,190.466500,319.353623,589.743946
1,10.274655,19.053324,33.004086,75.182165,35.923086,0,2,6,0,2700.775347,195.766325,339.105595,628.837531
2,10.329388,17.053652,31.626523,72.317361,35.654168,0,2,6,0,2578.415324,176.153783,326.682614,539.347728
3,10.167943,18.328607,33.213043,73.215120,35.576479,0,2,6,0,2604.736209,186.364223,337.708314,608.748804
4,10.121747,17.553675,34.435514,71.837578,35.993904,0,2,6,0,2585.714916,177.673859,348.547559,604.469827
5,10.293880,17.456601,34.523300,71.102144,35.686812,0,2,6,0,2537.408863,179.696164,355.378721,602.659487
6,10.513439,20.577635,35.446381,75.615619,35.986326,0,2,6,0,2721.128358,216.341709,372.663355,729.402712
7,10.335250,16.452334,34.961154,70.948938,35.794074,0,2,6,0,2539.551535,170.038978,361.332252,575.192572
8,10.581166,20.406577,35.877570,71.142555,35.910433,0,2,6,0,2554.759985,215.925376,379.626514,732.138392
9,10.518996,16.780796,34.829136,71.597053,35.605304,0,2,6,0,2549.234858,176.517127,366.367539,584.460625


In [14]:
y_train[:10]

,pm25_t+1,pm25_t+2,pm25_t+3,pm25_t+4,pm25_t+5,pm25_t+6
0,33.004086,31.626523,33.213043,34.435514,34.523300,35.446381
1,31.626523,33.213043,34.435514,34.523300,35.446381,34.961154
2,33.213043,34.435514,34.523300,35.446381,34.961154,35.877570
3,34.435514,34.523300,35.446381,34.961154,35.877570,34.829136
4,34.523300,35.446381,34.961154,35.877570,34.829136,35.027099
5,35.446381,34.961154,35.877570,34.829136,35.027099,31.897059
6,34.961154,35.877570,34.829136,35.027099,31.897059,33.163362
7,35.877570,34.829136,35.027099,31.897059,33.163362,35.221457
8,34.829136,35.027099,31.897059,33.163362,35.221457,34.597436
9,35.027099,31.897059,33.163362,35.221457,34.597436,33.832250


In [15]:
df.columns

Index(['date', 'pm1', 'pm10', 'pm25', 'rh', 'temp', 'hour', 'day_of_week',
       'month', 'is_weekend', 'temp_x_rh', 'pm1_x_pm10', 'pm1_x_pm25',
       'pm10_x_pm25', 'pm25_t+1', 'pm25_t+2', 'pm25_t+3', 'pm25_t+4',
       'pm25_t+5', 'pm25_t+6'],
      dtype='object')

In [16]:
df.tail()

,date,pm1,pm10,pm25,rh,temp,hour,day_of_week,month,is_weekend,temp_x_rh,pm1_x_pm10,pm1_x_pm25,pm10_x_pm25,pm25_t+1,pm25_t+2,pm25_t+3,pm25_t+4,pm25_t+5,pm25_t+6
296341,2025-03-25 23:05:00+00:00,10.306731,15.921469,32.575195,71.509534,35.980509,23,1,3,0,2572.949432,164.098302,335.743781,518.644947,32.956039,35.689928,34.906417,34.077142,33.486693,33.229979
296342,2025-03-25 23:10:00+00:00,10.175454,18.203499,32.956039,72.208955,35.560817,23,1,3,0,2567.809461,185.228856,335.342643,599.915203,35.689928,34.906417,34.077142,33.486693,33.229979,35.941677
296343,2025-03-25 23:15:00+00:00,10.440616,17.418756,35.689928,73.651152,35.979522,23,1,3,0,2649.933247,181.862542,372.624831,621.674142,34.906417,34.077142,33.486693,33.229979,35.941677,32.443430
296344,2025-03-25 23:20:00+00:00,10.516851,19.215509,34.906417,73.175177,35.642104,23,1,3,0,2608.117283,202.086643,367.105581,670.744568,34.077142,33.486693,33.229979,35.941677,32.443430,33.554267
296345,2025-03-25 23:25:00+00:00,10.117945,20.124501,34.077142,71.000460,35.786191,23,1,3,0,2540.836022,203.618606,344.790668,685.785496,33.486693,33.229979,35.941677,32.443430,33.554267,35.387460


In [17]:
import xgboost as xgb
import mlflow.xgboost
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize XGBoost model for multi-output regression
model = xgb.XGBRegressor(
    objective="reg:squarederror",
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

# Train model
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate the mean absolute error for each forecast step
mae = mean_absolute_error(y_test, y_pred, multioutput="raw_values")
mse = mean_squared_error(y_test, y_pred, multioutput="raw_values")
r2 = r2_score(y_test, y_pred, multioutput="raw_values")

# Log model to MLflow
with mlflow.start_run():
    mlflow.log_param("model_type", "XGBoost")

    for i in range(forecast_horizon):
        mlflow.log_metric(f"mae_t_{i+1}", mae[i])
        mlflow.log_metric(f"mse_t_{i+1}", mse[i])
        mlflow.log_metric(f"r2_t_{i+1}", r2[i])

    # Log the model
    mlflow.xgboost.log_model(model, "model")

print("✅ XGBoost model logged in MLflow with additional metrics.")


e:\projects\datasys\venv\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [20:04:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/04/03 20:04:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ XGBoost model logged in MLflow with additional metrics.


In [18]:
import mlflow

print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: sqlite:///E:/projects/datasys/mlflow/mlflow.db


In [19]:
y_pred[:10]

array([[33.694176, 33.746456, 33.942062, 33.851917, 33.981667, 33.806   ],
       [33.764183, 33.766075, 33.865265, 33.798847, 33.82187 , 33.71327 ],
       [33.956238, 33.881413, 33.818626, 33.79876 , 33.780525, 33.77749 ],
       [33.808483, 33.786068, 33.804184, 33.78609 , 33.831196, 33.828182],
       [33.76555 , 33.77765 , 33.789886, 33.797356, 33.734467, 33.77418 ],
       [33.769432, 33.806267, 33.822166, 33.71809 , 33.755035, 33.784946],
       [33.742107, 33.640015, 33.635475, 33.822968, 33.89194 , 33.651005],
       [33.736206, 33.8423  , 33.92958 , 33.87234 , 33.81524 , 33.657436],
       [33.777916, 33.682636, 33.817886, 33.821312, 33.8554  , 33.80972 ],
       [33.883625, 33.68695 , 34.29875 , 33.010414, 33.710644, 33.976902]],
      dtype=float32)

In [20]:
X_test[:10]

,pm1,pm10,pm25,rh,temp,hour,day_of_week,month,is_weekend,temp_x_rh,pm1_x_pm10,pm1_x_pm25,pm10_x_pm25
237076,10.100731,19.726310,34.047517,74.865131,35.720834,4,6,9,1,2674.244942,199.250144,343.904800,671.631860
237077,10.515692,15.823706,35.801612,72.260370,35.603079,4,6,9,1,2572.691671,166.397217,376.478727,566.514177
237078,10.402996,17.354303,31.785158,71.312745,35.950271,4,6,9,1,2563.712556,180.536747,330.660883,551.609252
237079,10.266758,18.303956,33.197737,71.732697,35.899319,4,6,9,1,2575.154989,187.922293,340.833138,607.649925
237080,10.393975,20.555152,34.239825,73.990412,35.609355,4,6,9,1,2634.750813,213.649731,355.887874,703.804812
237081,10.153652,19.546890,31.974051,74.622957,35.893699,4,6,9,1,2678.493971,198.472317,324.653389,624.993254
237082,10.374640,15.761497,31.704778,73.414157,35.752874,4,6,9,1,2624.767102,163.519860,328.925665,499.714764
237083,10.171255,15.800792,33.087241,75.684088,35.883538,4,6,9,1,2715.812801,160.713880,336.538753,522.804606
237084,10.396833,18.552614,36.178870,74.567574,35.638815,5,6,9,1,2657.499986,192.888423,376.145653,671.212623
237085,10.122992,20.092114,35.263473,70.805674,35.509465,5,6,9,1,2514.271604,203.392301,356.971836,708.517716


In [33]:
df.columns

Index(['date', 'pm1', 'pm10', 'pm25', 'rh', 'temp', 'hour', 'day_of_week',
       'month', 'is_weekend', 'temp_x_rh', 'pm1_x_pm10', 'pm1_x_pm25',
       'pm10_x_pm25'],
      dtype='object')

In [10]:
df['location_id'] = "colombo"
df['is_weekend'] = df['is_weekend'].astype(bool)
df_selected = df[['location_id', 'date', 'pm1', 'pm10', 'pm25', 'rh', 'temp', 'hour', 'day_of_week',
       'month', 'is_weekend', 'temp_x_rh', 'pm1_x_pm10', 'pm1_x_pm25',
       'pm10_x_pm25']]

In [11]:
df_selected.head()

,location_id,date,pm1,pm10,pm25,rh,temp,hour,day_of_week,month,is_weekend,temp_x_rh,pm1_x_pm10,pm1_x_pm25,pm10_x_pm25
0,colombo,2022-06-01 00:00:00+00:00,10.155785,18.754484,31.445490,71.800804,35.776383,0,2,6,False,2568.773091,190.466500,319.353623,589.743946
1,colombo,2022-06-01 00:05:00+00:00,10.274655,19.053324,33.004086,75.182165,35.923086,0,2,6,False,2700.775347,195.766325,339.105595,628.837531
2,colombo,2022-06-01 00:10:00+00:00,10.329388,17.053652,31.626523,72.317361,35.654168,0,2,6,False,2578.415324,176.153783,326.682614,539.347728
3,colombo,2022-06-01 00:15:00+00:00,10.167943,18.328607,33.213043,73.215120,35.576479,0,2,6,False,2604.736209,186.364223,337.708314,608.748804
4,colombo,2022-06-01 00:20:00+00:00,10.121747,17.553675,34.435514,71.837578,35.993904,0,2,6,False,2585.714916,177.673859,348.547559,604.469827


In [12]:
df_selected = df_selected.dropna().reset_index(drop=True)

In [13]:
df_selected.head()

,location_id,date,pm1,pm10,pm25,rh,temp,hour,day_of_week,month,is_weekend,temp_x_rh,pm1_x_pm10,pm1_x_pm25,pm10_x_pm25
0,colombo,2022-06-01 00:00:00+00:00,10.155785,18.754484,31.445490,71.800804,35.776383,0,2,6,False,2568.773091,190.466500,319.353623,589.743946
1,colombo,2022-06-01 00:05:00+00:00,10.274655,19.053324,33.004086,75.182165,35.923086,0,2,6,False,2700.775347,195.766325,339.105595,628.837531
2,colombo,2022-06-01 00:10:00+00:00,10.329388,17.053652,31.626523,72.317361,35.654168,0,2,6,False,2578.415324,176.153783,326.682614,539.347728
3,colombo,2022-06-01 00:15:00+00:00,10.167943,18.328607,33.213043,73.215120,35.576479,0,2,6,False,2604.736209,186.364223,337.708314,608.748804
4,colombo,2022-06-01 00:20:00+00:00,10.121747,17.553675,34.435514,71.837578,35.993904,0,2,6,False,2585.714916,177.673859,348.547559,604.469827


In [37]:
df_selected.columns

Index(['location_id', 'date', 'pm1', 'pm10', 'pm25', 'rh', 'temp', 'hour',
       'day_of_week', 'month', 'is_weekend', 'temp_x_rh', 'pm1_x_pm10',
       'pm1_x_pm25', 'pm10_x_pm25'],
      dtype='object')

In [16]:
import psycopg2
import pandas as pd
from psycopg2.extras import execute_values
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Database Config
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")

# Connect to PostgreSQL
def create_connection():
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        return conn
    except psycopg2.Error as e:
        print(f"❌ Error connecting to database: {e}")
        return None

# Function to insert DataFrame into PostgreSQL
def insert_dataframe(df, table_name="air_quality_feature_store"):
    conn = create_connection()
    if conn is None:
        print("❌ Connection failed. Exiting.")
        return

    try:
        cursor = conn.cursor()

        # Convert DataFrame to list of tuples for insertion
        records = [tuple(row) for row in df.itertuples(index=False, name=None)]

        # Define the SQL query (column names should match the table schema)
        columns = ",".join(df.columns)
        insert_query = f"""
            INSERT INTO {table_name} ({columns})
            VALUES %s;
        """

        # Use execute_values for fast bulk insertion
        execute_values(cursor, insert_query, records)

        # Commit and close connection
        conn.commit()
        cursor.close()
        conn.close()

        print(f"✅ Successfully inserted {len(df)} records into {table_name}!")
    
    except psycopg2.Error as e:
        print(f"❌ Error inserting data: {e}")



In [17]:
insert_dataframe(df_selected)

✅ Successfully inserted 296352 records into air_quality_feature_store!


In [1]:
#  flask app test

In [ ]:
import requests

# Define column names (as expected by the model)
columns = [
    "pm1", "pm10", "pm25", "rh", "temp",
    "hour", "day_of_week", "month", "is_weekend",
    "temp_x_rh", "pm1_x_pm10", "pm1_x_pm25", "pm10_x_pm25"
]

# Define the first row of your test data
data_row = [
    10.155785, 18.754484, 31.445490, 71.800804, 35.776383,
    0, 2, 6, 0,
    2568.773091, 190.466500, 319.353623, 589.743946
]


# Prepare the request payload
payload = {
    "columns": columns,
    "data": [data_row] 
}
print(payload)

# Make POST request to the /predict endpoint
response = requests.post("http://localhost:5050/predict", json=payload)

# Display the result
if response.status_code == 200:
    print("✅ Prediction:", response.json())
else:
    print("❌ Error:", response.status_code, response.text)


{'columns': ['pm1', 'pm10', 'pm25', 'rh', 'temp', 'hour', 'day_of_week', 'month', 'is_weekend', 'temp_x_rh', 'pm1_x_pm10', 'pm1_x_pm25', 'pm10_x_pm25'], 'data': [[10.155785, 18.754484, 31.44549, 71.800804, 35.776383, 0, 2, 6, 0, 2568.773091, 190.4665, 319.353623, 589.743946]]}
✅ Prediction: {'prediction': [[33.824188232421875, 33.78282165527344, 33.67688751220703, 33.75773620605469, 33.71537780761719, 33.781707763671875]]}
